In [1]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [5]:
# Get images
X = []
for filename in os.listdir(r"C:\Users\risha\Documents\Project_X\Coloring-greyscale-images-master\Full-version\Train\\"):
    X.append(img_to_array(load_img(r"C:\Users\risha\Documents\Project_X\Coloring-greyscale-images-master\Full-version\Train\\"+filename)))
X = np.array(X, dtype=float)

# Set up train and test data
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

In [7]:
Xtrain.shape

(9, 256, 256, 3)

In [8]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [19]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.3,
        zoom_range=0.3,
        rotation_range=30,
        horizontal_flip=True)

# Generate training data
batch_size = 10
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

# Train model      
tensorboard = TensorBoard(log_dir="output/first_run")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=50, steps_per_epoch=10)

C:\Users\risha\AppData\Local\Temp\ipykernel_12548\3651549848.py:19: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=50, steps_per_epoch=10)


Epoch 1/50
10/10 [==============================] - 21s 2s/step - loss: 0.0056
Epoch 2/50
10/10 [==============================] - 20s 2s/step - loss: 0.0053
Epoch 3/50
10/10 [==============================] - 20s 2s/step - loss: 0.0055
Epoch 4/50
10/10 [==============================] - 21s 2s/step - loss: 0.0056
Epoch 5/50
10/10 [==============================] - 21s 2s/step - loss: 0.0055
Epoch 6/50
10/10 [==============================] - 21s 2s/step - loss: 0.0055
Epoch 7/50
10/10 [==============================] - 21s 2s/step - loss: 0.0053
Epoch 8/50
10/10 [==============================] - 23s 2s/step - loss: 0.0061
Epoch 9/50
10/10 [==============================] - 23s 2s/step - loss: 0.0054
Epoch 10/50
10/10 [==============================] - 22s 2s/step - loss: 0.0055
Epoch 11/50
10/10 [==============================] - 21s 2s/step - loss: 0.0053
Epoch 12/50
10/10 [==============================] - 22s 2s/step - loss: 0.0053
Epoch 13/50
10/10 [==============================

In [20]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [21]:
# Test images
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

1/1 [==============================] - 0s 95ms/step - loss: 0.0039
0.003907251171767712


In [22]:
color_me = []
for filename in os.listdir(r"C:\Users\risha\Documents\Project_X\Coloring-greyscale-images-master\Full-version\Test\\"):
    color_me.append(img_to_array(load_img(r"C:\Users\risha\Documents\Project_X\Coloring-greyscale-images-master\Full-version\Test\\"+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))


1/1 [==============================] - 0s 408ms/step


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
